In [4]:
!pip install Flask

In [2]:
from flask import Flask

app = Flask(__name__)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\jose1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

Clientes en la base de datos: [<__main__.Cliente object at 0x0000013F9DAA5790>, <__main__.Cliente object at 0x0000013F9DAA5C10>]
Peluqueros en la base de datos: [<__main__.Peluquero object at 0x0000013F9DAA5B80>, <__main__.Peluquero object at 0x0000013F9DAA5700>]
Citas en la base de datos: [<__main__.Cita object at 0x0000013F9DAA5EE0>, <__main__.Cita object at 0x0000013F9DAA5FA0>]


Cita para Cliente1 con Peluquero1 el 2024-01-15 10:00:00 para Corte
Cita para Cliente2 con Peluquero2 el 2024-01-15 11:00:00 para Peinado


!pip install psycopg2-binary

In [ ]:
# MI PROGRAMA SE DIVIDE EN DOS PARTES UNA QUE SE CONECTA A LA BASE DE DATOS EN ESTE CASO POSTGRES 
# Y OTRA ES MUESTRA  MENU Y SUS OPCIONES FUNCIONA CORRECTAMENTE
# HAY OTRA PARTE DONDE SE CONECTA POR HTTP JSON Y ES COMSUMIDA POR REACT DONDE MUESTRA UN MENU, UN TABLERO
import psycopg2
from psycopg2 import sql
from typing import List 
from datetime import datetime  # Importa datetime desde el módulo correcto

# Configurar la conexión a la base de datos
try:
    connection = psycopg2.connect(
        dbname="salon_peluqueria",
        user="postgres",  # en este caso postgres
        password="12345",  # la que tenia mi clave
        host="localhost",
        port="5432"
    )
    print("Conexión exitosa a la base de datos")
except Exception as e:
    print("Error al conectar a la base de datos:", e)

def agregar_cliente(nombre: str, telefono: str, email: str):
    try:
        with connection.cursor() as cursor:
            cursor.execute(
                sql.SQL("INSERT INTO clientes (nombre, telefono, email) VALUES (%s, %s, %s) RETURNING *"),
                (nombre, telefono, email)
            )
            cliente = cursor.fetchone()
            connection.commit()
        print("Cliente agregado:", cliente)
        return cliente
    except Exception as e:
        print("Error al agregar cliente:", e)

def agregar_peluquero(nombre: str, servicios: List[str]):
    with connection.cursor() as cursor:
        cursor.execute(
            sql.SQL("INSERT INTO peluqueros (nombre, servicios) VALUES (%s, %s) RETURNING *"),
            (nombre, servicios)
        )
        peluquero = cursor.fetchone()
        connection.commit()
    return peluquero

def agregar_cita(id_cliente: int, id_peluquero: int, servicio: str, fecha: datetime):
    with connection.cursor() as cursor:
        cursor.execute(
            sql.SQL("INSERT INTO citas (id_cliente, id_peluquero, servicio, fecha) VALUES (%s, %s, %s, %s) RETURNING *"),
            (id_cliente, id_peluquero, servicio, fecha)
        )
        cita = cursor.fetchone()
        connection.commit()
    return cita


In [95]:

def menu_principal():
    while True:
        print("\n---- Menú Principal ----")
        print("1. Agregar Cliente")
        print("2. Agregar Peluquero")
        print("3. Agregar Cita")
        print("4. Salir")

        opcion = input("Seleccione una opción (1-4): ")

        if opcion == '1':
            agregar_cliente_interfaz()
        elif opcion == '2':
            agregar_peluquero_interfaz()
        elif opcion == '3':
            agregar_cita_interfaz()
        elif opcion == '4':
            print("Saliendo del programa. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

def agregar_cliente_interfaz():
    print("\n---- Agregar Cliente ----")
    nombre = input("Nombre: ")
    telefono = input("Teléfono: ")
    email = input("Email: ")

    cliente_agregado = agregar_cliente(nombre, telefono, email)

    if cliente_agregado:
        print("Cliente agregado con éxito:", cliente_agregado)
    else:
        print("Error al agregar cliente.")

def agregar_peluquero_interfaz():
    print("\n---- Agregar Peluquero ----")
    nombre = input("Nombre: ")
    servicios = input("Servicios (separados por comas): ").split(',')

    peluquero_agregado = agregar_peluquero(nombre, servicios)

    if peluquero_agregado:
        print("Peluquero agregado con éxito:", peluquero_agregado)
    else:
        print("Error al agregar peluquero.")

def agregar_cita_interfaz():
    print("\n---- Agregar Cita ----")
    id_cliente = int(input("ID del Cliente: "))
    id_peluquero = int(input("ID del Peluquero: "))
    servicio = input("Servicio: ")
    fecha_str = input("Fecha y hora (YYYY-MM-DD HH:MM): ")

    try:
        fecha = datetime.strptime(fecha_str, "%Y-%m-%d %H:%M")
        cita_agregada = agregar_cita(id_cliente, id_peluquero, servicio, fecha)

        if cita_agregada:
            print("Cita agregada con éxito:", cita_agregada)
        else:
            print("Error al agregar cita.")
    except ValueError:
        print("Formato de fecha incorrecto. Use YYYY-MM-DD HH:MM.")

# Play el programa
menu_principal()
# este funciona OK


---- Menú Principal ----
1. Agregar Cliente
2. Agregar Peluquero
3. Agregar Cita
4. Salir


KeyboardInterrupt: Interrupted by user

In [81]:
import psycopg2
from psycopg2 import sql
from typing import List 
from datetime import datetime, timedelta

# Configurar la conexión a la base de datos
try:
    connection = psycopg2.connect(
        dbname="salon_peluqueria",
        user="postgres",  # Reemplaza con tu nombre de usuario
        password="12345",  # Reemplaza con tu contraseña
        host="localhost",
        port="5432"
    )
    print("Conexión exitosa a la base de datos")
except Exception as e:
    print("Error al conectar a la base de datos:", e)

def agregar_cliente(nombre: str, telefono: str, email: str):
    try:
        with connection.cursor() as cursor:
            cursor.execute(
                sql.SQL("INSERT INTO clientes (nombre, telefono, email) VALUES (%s, %s, %s) RETURNING *"),
                (nombre, telefono, email)
            )
            cliente = cursor.fetchone()
            connection.commit()
        print("Cliente agregado:", cliente)
        return cliente
    except Exception as e:
        print("Error al agregar cliente:", e)

def agregar_peluquero(nombre: str, servicios: List[str]):
    with connection.cursor() as cursor:
        cursor.execute(
            sql.SQL("INSERT INTO peluqueros (nombre, servicios) VALUES (%s, %s) RETURNING *"),
            (nombre, servicios)
        )
        peluquero = cursor.fetchone()
        connection.commit()
    return peluquero

def agregar_cita(id_cliente: int, id_peluquero: int, servicio: str, fecha: datetime):
    # Verificar la disponibilidad antes de agregar la cita
    if verificar_disponibilidad(id_peluquero, fecha):
        with connection.cursor() as cursor:
            cursor.execute(
                sql.SQL("INSERT INTO citas (id_cliente, id_peluquero, servicio, fecha) VALUES (%s, %s, %s, %s) RETURNING *"),
                (id_cliente, id_peluquero, servicio, fecha)
            )
            cita = cursor.fetchone()
            connection.commit()
        print("Cita agregada:", cita)
        return cita
    else:
        print("No se pudo agregar la cita debido a la falta de disponibilidad.")
        return None

def verificar_disponibilidad(id_peluquero: int, fecha: datetime):
    # Definir la ventana de tiempo para verificar la disponibilidad (por ejemplo, 1 hora)
    ventana_tiempo = timedelta(hours=1)

    # Calcular el rango de tiempo para buscar citas
    inicio_ventana = fecha - ventana_tiempo
    fin_ventana = fecha + ventana_tiempo

    with connection.cursor() as cursor:
        # Consultar citas del peluquero en el rango de tiempo
        cursor.execute(
            sql.SQL("SELECT * FROM citas WHERE id_peluquero = %s AND fecha BETWEEN %s AND %s"),
            (id_peluquero, inicio_ventana, fin_ventana)
        )
        citas = cursor.fetchall()

    # Verificar si hay alguna cita programada en el rango de tiempo
    if citas:
        print("El peluquero ya tiene una cita programada en ese horario.")
        return False
    else:
        print("El peluquero está disponible en el horario seleccionado.")
        return True


Conexión exitosa a la base de datos


In [82]:
import psycopg2
from psycopg2 import sql
from typing import List
from datetime import datetime, timedelta

# Configurar la conexión a la base de datos
try:
    connection = psycopg2.connect(
        dbname="salon_peluqueria",
        user="postgres",
        password="12345",
        host="localhost",
        port="5432"
    )
    print("Conexión exitosa a la base de datos")
except Exception as e:
    print("Error al conectar a la base de datos:", e)

def mostrar_menu():
    print("\nMenú Principal:")
    print("1. Agregar Cliente")
    print("2. Agregar Peluquero")
    print("3. Agregar Cita")
    print("4. Verificar Disponibilidad")
    print("5. Salir")

def main():
    while True:
        mostrar_menu()
        opcion = input("Seleccione una opción (1-5): ")

        if opcion == "1":
            # Lógica para agregar cliente
            pass
        elif opcion == "2":
            # Lógica para agregar peluquero
            pass
        elif opcion == "3":
            # Lógica para agregar cita
            pass
        elif opcion == "4":
            # Lógica para verificar disponibilidad
            pass
        elif opcion == "5":
            print("Saliendo del programa. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, seleccione una opción del 1 al 5.")

if __name__ == "__main__":
    main()


Conexión exitosa a la base de datos

Menú Principal:
1. Agregar Cliente
2. Agregar Peluquero
3. Agregar Cita
4. Verificar Disponibilidad
5. Salir


KeyboardInterrupt: Interrupted by user

In [ ]:
import psycopg2
from psycopg2 import sql
from typing import List 
from datetime import datetime, timedelta

# Configurar la conexión a la base de datos
try:
    connection = psycopg2.connect(
        dbname="salon_peluqueria",
        user="postgres",  # Reemplaza con tu nombre de usuario
        password="12345",  # Reemplaza con tu contraseña
        host="localhost",
        port="5432"
    )
    print("Conexión exitosa a la base de datos")
except Exception as e:
    print("Error al conectar a la base de datos:", e)

def agregar_cliente(nombre: str, telefono: str, email: str):
    try:
        with connection.cursor() as cursor:
            cursor.execute(
                sql.SQL("INSERT INTO clientes (nombre, telefono, email) VALUES (%s, %s, %s) RETURNING *"),
                (nombre, telefono, email)
            )
            cliente = cursor.fetchone()
            connection.commit()
        print("Cliente agregado:", cliente)
        return cliente
    except Exception as e:
        print("Error al agregar cliente:", e)

def agregar_peluquero(nombre: str, servicios: List[str]):
    with connection.cursor() as cursor:
        cursor.execute(
            sql.SQL("INSERT INTO peluqueros (nombre, servicios) VALUES (%s, %s) RETURNING *"),
            (nombre, servicios)
        )
        peluquero = cursor.fetchone()
        connection.commit()
    return peluquero

def agregar_cita(id_cliente: int, id_peluquero: int, servicio: str, fecha: datetime):
    # Verificar la disponibilidad antes de agregar la cita
    if verificar_disponibilidad(id_peluquero, fecha):
        with connection.cursor() as cursor:
            cursor.execute(
                sql.SQL("INSERT INTO citas (id_cliente, id_peluquero, servicio, fecha) VALUES (%s, %s, %s, %s) RETURNING *"),
                (id_cliente, id_peluquero, servicio, fecha)
            )
            cita = cursor.fetchone()
            connection.commit()
        print("Cita agregada:", cita)
        return cita
    else:
        print("No se pudo agregar la cita debido a la falta de disponibilidad.")
        return None

def verificar_disponibilidad(id_peluquero: int, fecha: datetime):
    # Definir la ventana de tiempo para verificar la disponibilidad (por ejemplo, 1 hora)
    ventana_tiempo = timedelta(hours=1)

    # Calcular el rango de tiempo para buscar citas
    inicio_ventana = fecha - ventana_tiempo
    fin_ventana = fecha + ventana_tiempo

    with connection.cursor() as cursor:
        # Consultar citas del peluquero en el rango de tiempo
        cursor.execute(
            sql.SQL("SELECT * FROM citas WHERE id_peluquero = %s AND fecha BETWEEN %s AND %s"),
            (id_peluquero, inicio_ventana, fin_ventana)
        )
        citas = cursor.fetchall()

    # Verificar si hay alguna cita programada en el rango de tiempo
    if citas:
        print("El peluquero ya tiene una cita programada en ese horario.")
        return False
    else:
        print("El peluquero está disponible en el horario seleccionado.")
        return True


In [90]:
from datetime import datetime

def menu_principal():
    while True:
        print("\n---- Menú Principal ----")
        print("1. Agregar Cliente")
        print("2. Agregar Peluquero")
        print("3. Agregar Cita")
        print("4. Salir")

        opcion = input("Seleccione una opción (1-4): ")

        if opcion == '1':
            agregar_cliente_interfaz()
        elif opcion == '2':
            agregar_peluquero_interfaz()
        elif opcion == '3':
            agregar_cita_interfaz()
        elif opcion == '4':
            print("Saliendo del programa. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

def agregar_cliente_interfaz():
    print("\n---- Agregar Cliente ----")
    nombre = input("Nombre: ")
    telefono = input("Teléfono: ")
    email = input("Email: ")

    cliente_agregado = agregar_cliente(nombre, telefono, email)

    if cliente_agregado:
        print("Cliente agregado con éxito:", cliente_agregado)
    else:
        print("Error al agregar cliente.")

def agregar_peluquero_interfaz():
    print("\n---- Agregar Peluquero ----")
    nombre = input("Nombre: ")
    servicios = input("Servicios (separados por comas): ").split(',')

    peluquero_agregado = agregar_peluquero(nombre, servicios)

    if peluquero_agregado:
        print("Peluquero agregado con éxito:", peluquero_agregado)
    else:
        print("Error al agregar peluquero.")

def agregar_cita_interfaz():
    print("\n---- Agregar Cita ----")
    id_cliente = int(input("ID del Cliente: "))
    id_peluquero = int(input("ID del Peluquero: "))
    servicio = input("Servicio: ")
    fecha_str = input("Fecha y hora (YYYY-MM-DD HH:MM): ")

    try:
        fecha = datetime.strptime(fecha_str, "%Y-%m-%d %H:%M")
        # Verificar disponibilidad antes de agregar la cita
        if verificar_disponibilidad(id_peluquero, fecha):
        cita_agregada = agregar_cita(id_cliente, id_peluquero, servicio, fecha)

        if cita_agregada:
            print("Cita agregada con éxito:", cita_agregada)
        else:
            print("Error al agregar cita.")
        else:
            print("El peluquero no está disponible en el horario seleccionado.")
            
    except ValueError:
        print("Formato de fecha incorrecto. Use YYYY-MM-DD HH:MM.")

# Ejecutar el programa
menu_principal()
# este funciona

IndentationError: expected an indented block (<ipython-input-90-e495cc146fad>, line 61)

In [88]:
from datetime import datetime

def menu_principal():
    while True:
        print("\n---- Menú Principal ----")
        print("1. Agregar Cliente")
        print("2. Agregar Peluquero")
        print("3. Agregar Cita")
        print("4. Salir")

        opcion = input("Seleccione una opción (1-4): ")

        if opcion == '1':
            agregar_cliente_interfaz()
        elif opcion == '2':
            agregar_peluquero_interfaz()
        elif opcion == '3':
            agregar_cita_interfaz()
        elif opcion == '4':
            print("Saliendo del programa. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

def agregar_cliente_interfaz():
    print("\n---- Agregar Cliente ----")
    nombre = input("Nombre: ")
    telefono = input("Teléfono: ")
    email = input("Email: ")

    cliente_agregado = agregar_cliente(nombre, telefono, email)

    if cliente_agregado:
        print("Cliente agregado con éxito:", cliente_agregado)
    else:
        print("Error al agregar cliente.")

def agregar_peluquero_interfaz():
    print("\n---- Agregar Peluquero ----")
    nombre = input("Nombre: ")
    servicios = input("Servicios (separados por comas): ").split(',')

    peluquero_agregado = agregar_peluquero(nombre, servicios)

    if peluquero_agregado:
        print("Peluquero agregado con éxito:", peluquero_agregado)
    else:
        print("Error al agregar peluquero.")

def agregar_cita_interfaz():
    print("\n---- Agregar Cita ----")
    id_cliente = int(input("ID del Cliente: "))
    id_peluquero = int(input("ID del Peluquero: "))
    servicio = input("Servicio: ")
    fecha_str = input("Fecha y hora (YYYY-MM-DD HH:MM): ")

    try:
        fecha = datetime.strptime(fecha_str, "%Y-%m-%d %H:%M")
        # Verificar disponibilidad antes de agregar la cita
        if verificar_disponibilidad(id_peluquero, fecha):
        cita_agregada = agregar_cita(id_cliente, id_peluquero, servicio, fecha)

        if cita_agregada:
            print("Cita agregada con éxito:", cita_agregada)
        else:
            print("Error al agregar cita.")
        else:
            print("El peluquero no está disponible en el horario seleccionado.")
            
    except ValueError:
        print("Formato de fecha incorrecto. Use YYYY-MM-DD HH:MM.")

# Ejecutar el programa
menu_principal()
# este funciona

IndentationError: expected an indented block (<ipython-input-88-e495cc146fad>, line 61)